In [1]:
import re
import os
import json
from PIL import Image

import torch
import numpy as np
from tqdm import tqdm

import faiss
from sentence_transformers import SentenceTransformer
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer

root = "results/"
image_root = "F:/train_blocks"

In [3]:
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [4]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [5]:
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [6]:
def cap_post_process(caption):
    answers = []
    
    caps = re.split("CAP\s*\d*\s*[:=-]", caption)[1:]
    for cap in caps:
        if re.search("\d{2,}", cap):
            continue
        cap = cap.strip()
        cap = re.sub("^[\'\"]", "", cap)
        cap = re.sub("[\'\"]$", "", cap)
        answers.append(cap)
    return answers

def record_post_process(record):
    answers = {}
    
    for key, item in record.items():
        extracted_captions = []
        for caption in item["response"]:
            
            extracted_captions.extend(cap_post_process(caption))
        answers[key] = extracted_captions
    
    return answers

In [7]:
records = {}

for file_name in os.listdir(root):
    file_name_capture = re.search("records_(\d_[1-9]\d?).json", file_name)
    if file_name_capture:
        file_id = file_name_capture.group(1)
        
        with open(os.path.join(root, file_name), "r") as f:
            record = record_post_process(json.load(f))
        
        if len(record) == 0:
            continue
        records[file_id] = record

In [8]:
with open("results/sample.json", "r") as f:
    samples = json.load(f)

In [9]:
samples_keys = list(samples.keys())
records_keys = list(records.keys())

In [10]:
images = []

for key in samples_keys:
    file_path = os.path.join(image_root, f"{key}.png")
    if os.path.isfile(file_path):
        images.append(Image.open(file_path))

In [11]:
all_captions = []
samples_indices = []
records_indices = []

for key_idx, key in tqdm(enumerate(samples_keys)):
    sim_record = {}
    for record_id, record in records.items():
        sim_record[record_id] = record.get(key, [])
        
    lengths = [len(value) for value in sim_record.values()]
    print(lengths)
    if min(lengths) < 70:
        continue
        
    for record_key, captions in sim_record.items():
        record_idx = records_keys.index(record_key)
        all_captions.extend(captions)
        samples_indices.extend([key_idx] * len(captions))
        records_indices.extend([record_idx] * len(captions))

85it [00:00, 4857.76it/s]

[93, 82, 99, 94, 99, 93, 100, 92, 96, 98]
[99, 75, 100, 97, 99, 95, 100, 100, 96, 97]
[97, 70, 94, 93, 99, 99, 99, 88, 99, 99]
[96, 84, 97, 86, 86, 98, 91, 86, 92, 92]
[97, 86, 98, 94, 84, 88, 92, 83, 92, 92]
[95, 82, 97, 78, 90, 90, 95, 88, 81, 85]
[93, 94, 99, 76, 84, 98, 87, 84, 94, 95]
[90, 91, 97, 77, 82, 92, 92, 66, 87, 90]
[94, 79, 88, 89, 98, 96, 96, 87, 95, 87]
[95, 93, 105, 90, 96, 93, 92, 84, 94, 93]
[99, 94, 99, 97, 99, 92, 100, 94, 99, 92]
[97, 93, 94, 96, 100, 84, 93, 97, 94, 96]
[87, 79, 94, 95, 86, 77, 85, 65, 85, 98]
[97, 94, 100, 94, 97, 94, 92, 96, 94, 96]
[94, 86, 99, 89, 86, 97, 84, 76, 95, 96]
[97, 87, 100, 85, 90, 94, 89, 93, 99, 98]
[97, 80, 98, 85, 89, 98, 99, 89, 91, 98]
[98, 90, 89, 89, 96, 98, 95, 72, 91, 92]
[97, 92, 100, 97, 89, 96, 96, 88, 90, 96]
[91, 88, 96, 84, 97, 95, 94, 83, 93, 93]
[95, 94, 95, 95, 93, 94, 97, 87, 94, 94]
[98, 83, 88, 98, 99, 99, 96, 78, 92, 97]
[97, 92, 94, 83, 96, 95, 83, 81, 99, 96]
[98, 91, 100, 78, 94, 96, 91, 96, 91, 94]
[98, 

In [12]:
valid_index = [i for i, cap in enumerate(all_captions) if len(tokenizer.tokenize(cap)) <= 75]
valid_captions = [all_captions[i] for i in valid_index]

Token indices sequence length is longer than the specified maximum sequence length for this model (143 > 77). Running this sequence through the model will result in indexing errors


In [13]:
sample_idx = 0

valid_captions = [cap for i, cap in enumerate(valid_captions) if samples_indices[valid_index[i]] == sample_idx]
inputs = processor(text=valid_captions, images=images[sample_idx], return_tensors="pt", padding=True)

In [191]:
inputs["attention_mask"].shape

torch.Size([946, 36])

In [176]:
outpus = model(**inputs)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:75] data. DefaultCPUAllocator: not enough memory: you tried to allocate 10212335616 bytes. Buy new RAM!

In [142]:
outputs.logits_per_image[0]

tensor([[27.4078, 26.3227, 27.6466, 23.0295, 28.3662, 30.3864, 26.3013, 28.9818,
         27.6038, 27.2792, 28.0863, 30.0600, 24.6121, 29.9932, 30.1853, 25.2311,
         26.1103, 26.3962, 29.0061, 25.4596, 28.5840, 26.1389, 25.1561, 29.9808,
         27.4963, 26.7984, 29.4307, 29.9260, 27.9414, 26.7837, 24.6757, 28.0544,
         27.1940, 28.7220, 29.4552, 26.9890, 27.8348, 28.1866, 29.1882, 31.0258,
         24.0107, 23.9212, 19.5304, 24.2615, 22.3697, 27.4316, 26.2882, 27.7116,
         26.0907, 27.0374, 27.0597, 30.5158, 28.2198, 27.4131, 25.3951, 27.9326,
         26.1793, 23.3211, 29.2150, 25.4633, 24.6023, 22.5715, 24.1791, 24.4786,
         30.2361, 25.4230, 30.9019, 26.6131, 27.0803, 24.6719, 26.8904, 24.2388,
         26.7410, 29.2321, 25.7595, 27.6169, 29.5974, 25.8043, 31.3502, 29.6236,
         28.0659, 28.3439, 19.3412]], grad_fn=<TBackward>)

In [44]:
sim_records = {}

for key in tqdm(samples.keys()):
    sim_record = {}
    for record_id, record in records.items():
        captions = record.get(key, [])
        if len(captions) == 0:
            continue
        
        inputs = processor(text=captions, images=image, return_tensors="pt", padding=True)
        logits = model(**inputs).logits_per_image[0].detach().numpy()
        sorted_logits = logits[np.argsort(logits)[::-1]]
        sim_record[record_id] = sorted_logits
    
    lengths = [value.shape[0] for value in sim_record.values()]
    
    if min(lengths) < 16:
        continue
        
    sim_records[key] = sim_record

  9%|███████▊                                                                           | 8/85 [03:23<32:37, 25.42s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'F:/train_blocks\\1118.tif_52.png'

In [21]:
sorted_logits

array([31.350168, 31.025824, 30.901922, 30.515787, 30.386406, 30.236092,
       30.185266, 30.05999 , 29.993208, 29.98081 , 29.925995, 29.623564,
       29.597374, 29.45525 , 29.43065 , 29.232107, 29.214981, 29.188152,
       29.006079, 28.98182 , 28.721973, 28.583952, 28.366207, 28.343855,
       28.219797, 28.18656 , 28.086308, 28.065895, 28.054441, 27.941397,
       27.932589, 27.834793, 27.71163 , 27.646627, 27.616869, 27.603762,
       27.496286, 27.431639, 27.413095, 27.407822, 27.279213, 27.194002,
       27.080276, 27.059748, 27.037437, 26.989027, 26.89038 , 26.798405,
       26.78367 , 26.74097 , 26.613094, 26.39619 , 26.322653, 26.301308,
       26.28823 , 26.179272, 26.138912, 26.11029 , 26.09074 , 25.8043  ,
       25.759514, 25.46328 , 25.459604, 25.423025, 25.395092, 25.23114 ,
       25.156092, 24.675657, 24.671854, 24.612137, 24.602262, 24.47859 ,
       24.261518, 24.23877 , 24.179085, 24.01067 , 23.921213, 23.321102,
       23.029516, 22.571487, 22.369669, 19.530396],

In [64]:
captions_array[sorted_indices]

array(['Several buildings form a diagonal line across the image, ranging in size from large to small.',
       'Buildings are sparsely scattered across the image, the largest one stands out in the upper right corner.',
       'The buildings in the lower half of the image form a diagonal line',
       'Aerial view reveals an intricate assortment of buildings in diamond formation, highlighting the largest structure at the top-right corner.',
       'A cluster of buildings, ranging from small to large sizes, are evenly distributed throughout the image.',
       'A dominant building stands out among smaller structures in this aerial view',
       'The organized layout of the buildings in this aerial image is accentuated by the carefully positioned small cars in the corners.',
       'The buildings in this aerial image are arranged in a diagonal line and two small cars bookend the corners.',
       'The buildings in this aerial image are arranged in a diagonal line while the small cars fram

In [12]:
kmeans = faiss.Kmeans(embedding.shape[1], 5, niter=20, verbose=True)
kmeans.train(embedding[:30])

NameError: name 'embedding' is not defined

In [79]:
index = faiss.IndexFlatL2(embedding.shape[1])
index.add(embedding)
D, I = index.search(kmeans.centroids, 2)

In [40]:
records_10.keys()

dict_keys(['10.tif_18', '10.tif_19', '10.tif_31', '10.tif_32', '10.tif_33', '10.tif_34', '10.tif_35', '10.tif_45', '10.tif_46', '10.tif_47', '10.tif_48', '10.tif_60'])

In [80]:
for i in range(I.shape[0]):
    print([captions[i] for i in I[i, :]])

['Top and bottom right buildings cluster.', 'Clustered buildings in top right and center.']
['Mostly horizontal/vertical alignment', 'Unique damage patterns top left and center']
['Damaged buildings in top left and center top.', 'Damaged buildings are concentrated in the top left and center top, indicating a possible cause or target of the damage.']
['The construction sites in the center of the image are interposed by buildings in the top right and bottom right.', 'Buildings are dispersed across the image, except for one in the middle bottom where a construction site is located.']
['Hut or tent separate in top right', 'The hut or tent in the top right stands out amidst the permanent structures, suggesting a different social or economic status.']
